## Join
+ 여러 테이블을 연결해서 데이터를 검색하는 것
+ 테이블의 결합기준은 각 테이블에 존재하는 공통 속성임
+ 결합 유형 : inner join, outer join, self join
+ 데이터객체명.join(조인대상, 조인조건, 조인유형)

In [2]:
spark = SparkSession.builder.appName('emp').getOrCreate()

In [3]:
emp = spark.read.csv('data/employees.csv',header=True, inferSchema=True)
dept = spark.read.csv('data/departments.csv',header=True, inferSchema=True)

### 내부조인 : 공통 속성명이 다를 경우
* select * from emp e inner join dept d   
on e.DEPARTMENT_ID = d.DEPARTMENT_ID

In [3]:
join_cnd = emp.DEPARTMENT_ID == dept.DEPARTMENT_ID
empdept = emp.join(dept, join_cnd, 'inner')
empdept.show(5)

+-----------+----------+---------+--------+------------+----------+-------+-------+--------------+----------+-------------+-------------+---------------+----------+-----------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER| HIRE_DATE| JOB_ID| SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-----------+----------+---------+--------+------------+----------+-------+-------+--------------+----------+-------------+-------------+---------------+----------+-----------+
|        100|    Steven|     King|   SKING|515.123.4567|2003-06-17|AD_PRES|24000.0|          null|      null|           90|           90|      Executive|       100|       1700|
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568|2005-09-21|  AD_VP|17000.0|          null|       100|           90|           90|      Executive|       100|       1700|
|        102|       Lex|  De Haan| LDEHAAN|515.123.4569|2001-01-13|  AD_VP|17000.0|          null|       100|      

### 내부조인 : 공통 속성명이 같을 경우
* select * from emp e inner join dept d   
using (DEPARTMENT_ID)

In [4]:
empdept = emp.join(dept, 'DEPARTMENT_ID', 'inner')
empdept.show(5)

+-------------+-----------+----------+---------+--------+------------+----------+-------+-------+--------------+----------+---------------+----------+-----------+
|DEPARTMENT_ID|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER| HIRE_DATE| JOB_ID| SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+-----------+----------+---------+--------+------------+----------+-------+-------+--------------+----------+---------------+----------+-----------+
|           90|        100|    Steven|     King|   SKING|515.123.4567|2003-06-17|AD_PRES|24000.0|          null|      null|      Executive|       100|       1700|
|           90|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568|2005-09-21|  AD_VP|17000.0|          null|       100|      Executive|       100|       1700|
|           90|        102|       Lex|  De Haan| LDEHAAN|515.123.4569|2001-01-13|  AD_VP|17000.0|          null|       100|      Executive|       100|       1700|
|           60|       

### 외부조인
* full, left, right

In [4]:
empdept2 = emp.join(dept, 'DEPARTMENT_ID', 'outer')

empdept2.show(120)

+-------------+-----------+-----------+-----------+--------+------------------+----------+----------+-------+--------------+----------+--------------------+----------+-----------+
|DEPARTMENT_ID|EMPLOYEE_ID| FIRST_NAME|  LAST_NAME|   EMAIL|      PHONE_NUMBER| HIRE_DATE|    JOB_ID| SALARY|COMMISSION_PCT|MANAGER_ID|     DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+-----------+-----------+-----------+--------+------------------+----------+----------+-------+--------------+----------+--------------------+----------+-----------+
|         null|        178|  Kimberely|      Grant|  KGRANT|011.44.1644.429263|2007-05-24|    SA_REP| 7000.0|          0.15|       149|                null|      null|       null|
|           10|        200|   Jennifer|     Whalen| JWHALEN|      515.123.4444|2003-09-17|   AD_ASST| 4400.0|          null|       101|      Administration|       200|       1700|
|           20|        201|    Michael|  Hartstein|MHARTSTE|      515.123.5555|2004-02-17|    MK_MAN

In [ ]:
# 사원이 한 명도 없는 부서를 조회
# select * from emp e outer join dept d
# using DEPARTMENT_ID where e.FIRST_NAME is null

In [5]:
import pyspark.sql.functions as F

In [7]:
# 사원테이블에 있는 부서 번호
emp.select('DEPARTMENT_ID').distinct().orderBy('DEPARTMENT_ID').show()

+-------------+
|DEPARTMENT_ID|
+-------------+
|         null|
|           10|
|           20|
|           30|
|           40|
|           50|
|           60|
|           70|
|           80|
|           90|
|          100|
|          110|
+-------------+



In [8]:
# 부서테이블에 있는 부서 번호
dept.select('DEPARTMENT_ID').distinct().orderBy('DEPARTMENT_ID').show()

+-------------+
|DEPARTMENT_ID|
+-------------+
|           10|
|           20|
|           30|
|           40|
|           50|
|           60|
|           70|
|           80|
|           90|
|          100|
|          110|
|          120|
|          130|
|          140|
|          150|
|          160|
|          170|
|          180|
|          190|
|          200|
+-------------+
only showing top 20 rows



In [7]:
empdept2.where(empdept2.FIRST_NAME.isNull())\
    .select('DEPARTMENT_ID','DEPARTMENT_NAME').show()

+-------------+--------------------+
|DEPARTMENT_ID|     DEPARTMENT_NAME|
+-------------+--------------------+
|          120|            Treasury|
|          130|       Corporate Tax|
|          140|  Control And Credit|
|          150|Shareholder Services|
|          160|            Benefits|
|          170|       Manufacturing|
|          180|        Construction|
|          190|         Contracting|
|          200|          Operations|
|          210|          IT Support|
|          220|                 NOC|
|          230|         IT Helpdesk|
|          240|    Government Sales|
|          250|        Retail Sales|
|          260|          Recruiting|
|          270|             Payroll|
+-------------+--------------------+



In [8]:
# 부서에 소속되지 않은 사원을 조회
# select * from emp e outer join dept d
# using DEPARTMENT_ID where DEPARTMENT_ID is null
empdept2.where(empdept2.DEPARTMENT_ID.isNull())\
    .select('FIRST_NAME','LAST_NAME').show()

+----------+---------+
|FIRST_NAME|LAST_NAME|
+----------+---------+
| Kimberely|    Grant|
+----------+---------+



## 예제 데이터베이스 - 고객, 주문, 제품

In [18]:
cust = spark.read.csv('data/Customers.csv',header=True, inferSchema=True)
order = spark.read.csv('data/Orders.csv',header=True, inferSchema=True)
products = spark.read.csv('data/Products.csv',header=True, inferSchema=True)

In [20]:
cust.columns

['userid', 'name', 'age', 'grade', 'job', 'coins']

In [21]:
order.columns

['orderid', 'userid', 'prodid', 'amount', 'addr', 'orddate']

In [22]:
products.columns

['prodid', 'prodname', 'stock', 'price', 'maker']

In [15]:
cust.show()

+------+------+----+------+------+-----+
|userid|  name| age| grade|   job|coins|
+------+------+----+------+------+-----+
| apple|정소화|  20|  gold|  학생| 1000|
|banana|김선우|  25|   vip|간호사| 2500|
|carrot|고명석|  28|  gold|  교사| 4500|
| melon|성원용|  35|  gold|회사원| 5000|
|orange|김용욱|  22|silver|  학생|    0|
| peach|오형준|null|silver|  의사|  300|
|  pear|채광주|  31|silver|회사원|  500|
+------+------+----+------+------+-----+



In [23]:
order.show()

+-------+------+------+------+---------------+----------+
|orderid|userid|prodid|amount|           addr|   orddate|
+-------+------+------+------+---------------+----------+
|    o01| apple|   p03|    10|  서울시 마포구|2013-01-01|
|    o02| melon|   p01|     5|  인천시 계양구|2013-01-10|
|    o03|banana|   p06|    45|  경기도 부천시|2013-01-11|
|    o04|carrot|   p02|     8|  부산시 금정구|2013-02-01|
|    o05| melon|   p06|    36|  경기도 용인시|2013-02-20|
|    o06|banana|   p01|    19|충청북도 보은군|2013-03-02|
|    o07| apple|   p03|    22|서울시 영등포구|2013-03-15|
|    o08|  pear|   p02|    50|  강원도 춘천시|2013-04-10|
|    o09|banana|   p04|    15|전라남도 목포시|2013-04-11|
|    o10|carrot|   p03|    20|  경기도 안양시|2013-05-22|
+-------+------+------+------+---------------+----------+



In [24]:
products.show()

+------+----------+-----+-----+--------+
|prodid|  prodname|stock|price|   maker|
+------+----------+-----+-----+--------+
|   p01|  그냥만두| 5000| 4500|대한식품|
|   p02|  매운쫄면| 2500| 5500|민국푸드|
|   p03|  쿵떡파이| 3600| 2600|한빛제과|
|   p04|맛난초콜렛| 1250| 2500|한빛제과|
|   p05|  얼큰라면| 2200| 1200|대한식품|
|   p06|  통통우동| 1000| 1550|민국푸드|
|   p07|달콤비스켓| 1650| 1500|한빛제과|
+------+----------+-----+-----+--------+



In [29]:
# 당근carrot 고객이 주문한 상품의 가격은 무엇인지 조회하세요
orderproducts = order.join(products, 'prodid', 'inner')
orderproducts.show(5)

+------+-------+------+------+-------------+----------+--------+-----+-----+--------+
|prodid|orderid|userid|amount|         addr|   orddate|prodname|stock|price|   maker|
+------+-------+------+------+-------------+----------+--------+-----+-----+--------+
|   p03|    o01| apple|    10|서울시 마포구|2013-01-01|쿵떡파이| 3600| 2600|한빛제과|
|   p01|    o02| melon|     5|인천시 계양구|2013-01-10|그냥만두| 5000| 4500|대한식품|
|   p06|    o03|banana|    45|경기도 부천시|2013-01-11|통통우동| 1000| 1550|민국푸드|
|   p02|    o04|carrot|     8|부산시 금정구|2013-02-01|매운쫄면| 2500| 5500|민국푸드|
|   p06|    o05| melon|    36|경기도 용인시|2013-02-20|통통우동| 1000| 1550|민국푸드|
+------+-------+------+------+-------------+----------+--------+-----+-----+--------+
only showing top 5 rows



In [33]:
orderproducts.where(orderproducts.userid == 'carrot')\
    .select('prodname','price').show()

+--------+-----+
|prodname|price|
+--------+-----+
|매운쫄면| 5500|
|쿵떡파이| 2600|
+--------+-----+



In [32]:
# 주문을 한번도 하지 않은 고객의 이름, 등급을 조회하세요
ordercust = order.join(cust, 'userid', 'outer')
ordercust.show(20)

+------+-------+------+------+---------------+----------+------+----+------+------+-----+
|userid|orderid|prodid|amount|           addr|   orddate|  name| age| grade|   job|coins|
+------+-------+------+------+---------------+----------+------+----+------+------+-----+
| apple|    o01|   p03|    10|  서울시 마포구|2013-01-01|정소화|  20|  gold|  학생| 1000|
| apple|    o07|   p03|    22|서울시 영등포구|2013-03-15|정소화|  20|  gold|  학생| 1000|
|banana|    o03|   p06|    45|  경기도 부천시|2013-01-11|김선우|  25|   vip|간호사| 2500|
|banana|    o06|   p01|    19|충청북도 보은군|2013-03-02|김선우|  25|   vip|간호사| 2500|
|banana|    o09|   p04|    15|전라남도 목포시|2013-04-11|김선우|  25|   vip|간호사| 2500|
|carrot|    o04|   p02|     8|  부산시 금정구|2013-02-01|고명석|  28|  gold|  교사| 4500|
|carrot|    o10|   p03|    20|  경기도 안양시|2013-05-22|고명석|  28|  gold|  교사| 4500|
| melon|    o02|   p01|     5|  인천시 계양구|2013-01-10|성원용|  35|  gold|회사원| 5000|
| melon|    o05|   p06|    36|  경기도 용인시|2013-02-20|성원용|  35|  gold|회사원| 5000|
|orange|   null|  null|  nu

In [34]:
ordercust.where(ordercust.orderid.isNull())\
    .select('userid','name', 'grade').show()

+------+------+------+
|userid|  name| grade|
+------+------+------+
|orange|김용욱|silver|
| peach|오형준|silver|
+------+------+------+



In [35]:
# 주문이 한번도 되지 않은 제품이름, 제조업체를 조회하세요
orderproducts2 = order.join(products, 'prodid', 'outer')
orderproducts2.show(20)

+------+-------+------+------+---------------+----------+----------+-----+-----+--------+
|prodid|orderid|userid|amount|           addr|   orddate|  prodname|stock|price|   maker|
+------+-------+------+------+---------------+----------+----------+-----+-----+--------+
|   p01|    o02| melon|     5|  인천시 계양구|2013-01-10|  그냥만두| 5000| 4500|대한식품|
|   p01|    o06|banana|    19|충청북도 보은군|2013-03-02|  그냥만두| 5000| 4500|대한식품|
|   p02|    o04|carrot|     8|  부산시 금정구|2013-02-01|  매운쫄면| 2500| 5500|민국푸드|
|   p02|    o08|  pear|    50|  강원도 춘천시|2013-04-10|  매운쫄면| 2500| 5500|민국푸드|
|   p03|    o01| apple|    10|  서울시 마포구|2013-01-01|  쿵떡파이| 3600| 2600|한빛제과|
|   p03|    o07| apple|    22|서울시 영등포구|2013-03-15|  쿵떡파이| 3600| 2600|한빛제과|
|   p03|    o10|carrot|    20|  경기도 안양시|2013-05-22|  쿵떡파이| 3600| 2600|한빛제과|
|   p04|    o09|banana|    15|전라남도 목포시|2013-04-11|맛난초콜렛| 1250| 2500|한빛제과|
|   p05|   null|  null|  null|           null|      null|  얼큰라면| 2200| 1200|대한식품|
|   p06|    o03|banana|    45|  경기도 부천시|2013

In [37]:
orderproducts2.where(orderproducts2.orderid.isNull())\
    .select('prodname', 'maker').show()

+----------+--------+
|  prodname|   maker|
+----------+--------+
|  얼큰라면|대한식품|
|달콤비스켓|한빛제과|
+----------+--------+



In [38]:
# 테이블 3개 조인하기
ordercustproducts = ordercust.join(products, 'prodid', 'inner')
ordercustproducts.show(5)

+------+------+-------+------+-------------+----------+------+---+-----+------+-----+--------+-----+-----+--------+
|prodid|userid|orderid|amount|         addr|   orddate|  name|age|grade|   job|coins|prodname|stock|price|   maker|
+------+------+-------+------+-------------+----------+------+---+-----+------+-----+--------+-----+-----+--------+
|   p03| apple|    o01|    10|서울시 마포구|2013-01-01|정소화| 20| gold|  학생| 1000|쿵떡파이| 3600| 2600|한빛제과|
|   p01| melon|    o02|     5|인천시 계양구|2013-01-10|성원용| 35| gold|회사원| 5000|그냥만두| 5000| 4500|대한식품|
|   p06|banana|    o03|    45|경기도 부천시|2013-01-11|김선우| 25|  vip|간호사| 2500|통통우동| 1000| 1550|민국푸드|
|   p02|carrot|    o04|     8|부산시 금정구|2013-02-01|고명석| 28| gold|  교사| 4500|매운쫄면| 2500| 5500|민국푸드|
|   p06| melon|    o05|    36|경기도 용인시|2013-02-20|성원용| 35| gold|회사원| 5000|통통우동| 1000| 1550|민국푸드|
+------+------+-------+------+-------------+----------+------+---+-----+------+-----+--------+-----+-----+--------+
only showing top 5 rows

